In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/hivep8
!hdfs dfs -ls /tmp/
!hdfs dfs -copyFromLocal tbl0.csv /tmp/hivep8/
!hdfs dfs -copyFromLocal tbl1.csv /tmp/hivep8/

Found 2 items
drwxrwx---   - root supergroup          0 2020-02-11 03:08 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-11 03:10 /tmp/hive
Found 3 items
drwxrwx---   - root supergroup          0 2020-02-11 03:08 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-11 03:10 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-11 03:11 /tmp/hivep8


In [3]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 24.118 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.724 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.906 seconds


In [4]:
%%hive
DROP TABLE IF EXISTS pt8;
CREATE TABLE pt8
AS
SELECT c2, SUM(numero)
FROM tbl0 LATERAL VIEW explode(c6) adTable AS letra,numero
GROUP BY c2;

INSERT OVERWRITE DIRECTORY '/tmp/hivep8'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
SELECT * FROM pt8;

DROP TABLE IF EXISTS pt8;
OK
Time taken: 0.023 seconds
CREATE TABLE pt8
AS
SELECT c2, SUM(numero)
FROM tbl0 LATERAL VIEW explode(c6) adTable AS letra,numero
GROUP BY c2;
Query ID = root_20200211031439_6b634dff-e625-471a-8ee4-532bb4a7f0a9
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1581390426457_0001, Tracking URL = http://714a45eef6db:8088/proxy/application_1581390426457_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1581390426457_0001
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-11 03:15:11,397 Stage-1 map = 0%,  reduce = 0%
2020-02-11 03:15:22,960 Stage-1 ma

In [5]:
!hdfs dfs -cat /tmp/hivep8/000000_0 

A,20
B,200
C,130
D,220


In [11]:
!hdfs dfs -copyToLocal /tmp/hivep8/000000_0 output;

In [8]:
%%hive
DROP TABLE tbl0;
DROP TABLE pt8;

DROP TABLE tbl0;
FAILED: ParseException line 2:0 missing EOF at 'DROP' near 'pt8'
DROP TABLE pt8;
OK
Time taken: 0.209 seconds


In [9]:
!rm *.csv *.log

rm: cannot remove 'derby.log': Text file busy


In [10]:
%hive_quit